# Basic Calcium Imaging Pipeline Using PCA-ICA
### Example with 4 rat videos
Importing Libraries and Videos for Data Analysis:

In [1]:
import os  # operating system
from glob import glob
from pathlib import Path  # to work with directories

import isx  # Inscopix API
import matplotlib.pyplot as plt  # data visualization
import numpy as np  # to manipulate data matrix
import pandas as pd  # data analysis library
import seaborn as sns
from cgk_calcium_tools import isx_files_handler #, plot_max_dff_and_cellmap_fh
from scipy import stats  # statistics library

sns.set_style("darkgrid")


outputsfolder = 'H:\\pipeline_example'  #it could be a list
mainfolder = '//CGK_NAS/Calcium Imaging Data/Rats/Calcium Imaging-MUSC Rats/Cohort 1' #it could be a list



data_subfolder = 'D1-838/D1-838_StressDay_10121' #it could be a list

# the videos are saved in folders based the animal id and the day when the videos were recorded
files_patterns = [
    "2021-10-01-11-27-26_video_trig_0.isxd",
    "2021-10-01-12-07-09_video_trig_0.isxd",
    "2021-10-01-13-42-25_video_trig_0.isxd",
    "2021-10-01-14-14-37_video_trig_0.isxd",
]

animal_sessions = { 
    #a key for each animal
    'D1-838': ['P1','S1','S2','P2'] #sessions in the same order as the files
}

#the next lines creates lists with the cell type and other metadata for each file using our naming convention and animal_sessions 
rats_ID = []
session_names = []
newlabels = []
cell_types = []
for cell_animal,sessions in animal_sessions.items():
    for session in sessions: 
        rats_ID.append(cell_animal[3:])
        session_names.append(session)
        newlabels.append(cell_animal + '_' + session)
        cell_types.append(cell_animal[:2])


fh = isx_files_handler(main_data_folder=mainfolder,
        outputsfolders=outputsfolder,
        data_subfolders=data_subfolder,
        files_patterns=files_patterns,
        processing_steps=["PP","TR", "BP", "MC"], recording_labels = newlabels)

video_len_seconds = int(29.5 * 60)  # setting the length of the video

AssertionError: None or multiple files found for \\CGK_NAS\Calcium Imaging Data\Rats\Calcium Imaging-MUSC Rats\Cohort 1\D1-838\D1-838_StressDay_10121\2021-10-01-11-27-26_video_trig_0.isxd.

In [ ]:
# create a table with video metadata
df_vd=fh.get_raw_movies_info().set_index("Full Path")

Checking the length of videos

In [ ]:
df_vd["Duration (s)"] > video_len_seconds

Trimming the videos to the same length 

In [ ]:
fh.de_interleave(False)
fh.run_step('PP')

This Cell preprocesses the videos and apply a spatial filter.
Preprocessing removes artifacts from the videos and reduces the size of the videos.
The Spatial filter removes the low and high frequency contents (removes noise).


In [ ]:
fh.run_step('TR')
fh.run_step('BP')
fh.run_step('MC')

This next applies motion correction to the videos removing any movement so the pixels align with the field of view in all frames.

In [ ]:
fh.create_dff()
fh.project_movie()

DeltaF/F0 normalizes the pixel values in the movies so that the variation in intensity is not due to spatial variation

The PCA-ICA algorithm identifies the spatial location of cells in the video and their associated activity using the principal and independent components of the video.

In [ ]:
fh.extract_cells('pca-ica')

Event Detection allows cells to be identified when they have an activity

This filters accepted cells based on different cell criteria

Shows the cellmaps for the accepted cells

In [ ]:
import warnings

warnings.filterwarnings("ignore")
fig = plt.figure(figsize=(10, 8))
for i, raw in enumerate(files_handler.rec_names):
    plt.subplot(2, 2, i + 1 + i // 4)
    cell_image = plot_max_dff_and_cellmap_fh(
        files_handler,
        idx=i,
        eqhist=True,
        cellsetname="pca-ica-cellset",
        status_list=["accepted"],
    )
    plt.title(newlabels[i])
    plt.xticks([])
    plt.yticks([])
fig.suptitle("Detected and Accepted Cells", fontsize=16)

plt.tight_layout()

Creates a plot of the cell traces

In [ ]:
plt.figure(figsize=(20, 12))
for fi, (name, cellset) in enumerate(
    zip(newlabels, files_handler.get_results_filenames("pca-ica-cellset", op="MC"))
):
    plt.subplot(2, 2, fi + 1 + fi // 4)
    cs = isx.CellSet.read(cellset)
    for n in range(cs.num_cells):
        if cs.get_cell_status(n) == "accepted":
            plt.plot(cs.get_cell_trace_data(n))
    plt.ylabel(name)

Creates the data to calculate average firing rate (events per minute) of the cells

In [ ]:
metadata = pd.DataFrame(
    {
        "file": files_handler.get_results_filenames("pca-ica-cellset", op="MC"),
        "animal_id": rats_ID,
        "session_name": session_names,
    }
)

events_df = []
for cellset, events in zip(
    files_handler.get_results_filenames("pca-ica-cellset", op="MC"),
    files_handler.get_results_filenames("pca-ica-event", op="MC"),
):
    ev = isx.EventSet.read(events)
    cs = isx.CellSet.read(cellset)
    for n in range(cs.num_cells):
        if cs.get_cell_status(n) == "accepted":
            events_df.append(
                pd.DataFrame(
                    {
                        "file": cellset,
                        "events_us": ev.get_cell_data(n)[0],
                        "cell_name": cs.get_cell_name(n),
                    }
                )
            )
events_df = pd.concat(events_df)
events_df["minutes"] = events_df["events_us"] / (60 * 1e6)
events_df = events_df.merge(metadata, on=["file"])
firing_rate = pd.DataFrame(
    events_df.groupby(["file", "cell_name"]).count()["events_us"]
)
firing_rate["#events/minute"] = firing_rate["events_us"] / (video_len_seconds / 60)
firing_rate = firing_rate.merge(metadata, on=["file"])

Example of bar graph to plot firing rate

In [ ]:
ax = sns.barplot(data=firing_rate, x="session_name", y="#events/minute", errorbar="se")
plt.title("Mean Firing Rate")

sns.stripplot(
    x="session_name", y="#events/minute", data=firing_rate, ax=ax,color='black',size=4
)

Calculates the cumulative events over each session

In [ ]:
times = np.linspace(
    0, video_len_seconds / 60, int(video_len_seconds * 2 / 60)
)  # 2 points per minute
cum_events_df = []
for file in events_df.file.unique():
    for cell in events_df[events_df.file == file]["cell_name"].unique():
        evs = np.sort(
            events_df[(events_df["cell_name"] == cell) * (events_df.file == file)][
                "minutes"
            ].values
        )
        calc_cum_events = np.vectorize(lambda x: sum(evs < x))
        cum_events = calc_cum_events(times)

        cum_events_df.append(
            pd.DataFrame(
                {
                    "file": file,
                    "cell_name": cell,
                    "Time (minutes)": times,
                    "cumulative events": cum_events.copy(),
                }
            )
        )


cum_events_df = pd.concat(cum_events_df)
cum_events_df = cum_events_df.merge(metadata, on=["file"])
display(cum_events_df)

Plots the event frequency of the cells during the recording

In [ ]:
sns.relplot(
    x="Time (minutes)",
    y="cumulative events",
    hue="session_name",
    data=cum_events_df,
    kind="line",
    facet_kws={"sharey": False},
    errorbar="se",
)